In [1]:
import tensorflow as tf
import os

In [2]:
os.chdir("../")

In [3]:
from pathlib import Path

In [4]:
from dataclasses import dataclass

@dataclass(frozen=True)
class EvaluationConfig:
    model_path : Path
    training_data: Path
    all_params: dict
    image_size: list
    batch_size: int

In [5]:
from deepClassifier.utils.common import read_yaml_file, create_directories, save_json
from deepClassifier.constant import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)
        create_directories([self.config.artifacts_root])
        
    
    def get_evaluation_config(self)->EvaluationConfig:
        evaluation_config = EvaluationConfig(
            model_path="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            all_params=self.params,
            image_size = self.params.IMAGE_SIZE,
            batch_size = self.params.BATCH
        )
        return evaluation_config

In [7]:
class EvaluationModel:
    def __init__(self,
                 config: EvaluationConfig):
        self.config = config
        
    def _test_data(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30)
        dataflow_kwargs = dict(
            target_size = self.config.image_size[:-1],
            batch_size = self.config.batch_size,
            interpolation='bilinear'
        )
        
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.test_data = test_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model = self.load_model(path=self.config.model_path)
        self._test_data()
        self.score = model.evaluate(self.test_data)
        
    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [8]:
config = ConfigurationManager()
evaluation_config = config.get_evaluation_config()
evaluation = EvaluationModel(config=evaluation_config)
evaluation.evaluation()

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1084s 2s/step - loss: 4.6207 - accuracy: 0.7285


In [9]:
evaluation.save_score()